In [10]:
#!git clone https://github.com/fany02656/Typicality_project.git

In [11]:
import numpy as np
import pandas as pd
import torch
from PIL import Image
from torchvision import transforms
import os
import matplotlib.pyplot as plt

In [12]:
import torch
model = torch.hub.load('pytorch/vision:v0.9.0', 'googlenet', pretrained=True)
model.eval()

Using cache found in C:\Users\Yi Wen/.cache\torch\hub\pytorch_vision_v0.9.0


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [14]:
from PIL import Image
from torchvision import transforms
import os

path='/content/drive/MyDrive/typicality/Typicality_project/All_Cropped'
images={}
directory=['Mountain', 'Beach', 'Mug', 'Banana', 
           'Car', 'Plane', 'Lighthouse', 'Church']

for dir in directory:
  images[dir]=[file for file in os.listdir(path+'/'+dir)]

In [ ]:
'''
from PIL import Image
from torchvision import transforms
import os

path='/content/drive/MyDrive/typicality/Typicality_project/Image'
images={}
directory=['Landscape_Scenery/Mountain', 'Landscape_Scenery/Beach', 'Daily_Object/Mug', 'Daily_Object/Banana', 
           'Vehicle/Car', 'Vehicle/Plane', 'Architecture/Lighthouse', 'Architecture/Church']

for dir in directory:
  images[dir]=[file for file in os.listdir(path+'/'+dir)]
'''

In [16]:
# #if windows
# from PIL import Image
# from torchvision import transforms
# import os

# # Load data: if Windows
# path='../Image/All_Cropped'

# images={}

# directory=['Mountain', 'Beach', 
#            'Mug', 'Banana', 
#            'Car', 'Plane', 
#            'Lighthouse', 'Church']

# for dir in directory:
#     images[dir]=[file for file in os.listdir(path+'/'+dir) if file.endswith(('jpeg', 'jpg'))]

In [17]:
for key in images.keys():
    print(key, ":", len(images[key]))

Mountain : 16
Beach : 16
Mug : 16
Banana : 16
Car : 16
Plane : 16
Lighthouse : 16
Church : 16


In [18]:
# read labels 
with open("../imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

# read labels to wordnet synsets
ltw = pd.read_json('../imagenet_label_to_wordnet_synset.json').T

In [19]:
ltw.columns

Index(['id', 'label', 'uri'], dtype='object')

In [20]:
preprocess = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
    transforms.ToTensor(),
])
all_images = []
for cate, filename in images.items():
    for f in filename:
        dir=os.path.join(path,cate,f)
        img=preprocess(Image.open(dir))
#         if img.shape[1] != 256 or img.shape[2] != 256:
#             print(img.shape)
#             plt.imshow(Image.open(dir))
#             plt.show()
        all_images.append(img)
all_images = torch.stack(all_images).numpy()
print(all_images.shape)
mean = np.mean(all_images, axis=(0,2,3))
std = np.std(all_images, axis=(0,2,3))

(128, 3, 256, 256)


In [21]:
mean

array([0.55640996, 0.5760121 , 0.56940985], dtype=float32)

In [22]:
std

array([0.3011326 , 0.28208593, 0.31378052], dtype=float32)

In [23]:
# preprocess = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean=mean, std=std),
# ])
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [24]:
#type prob cate+filename
all_img={'type':[], 'dir':[],'id_labels':[],'readable_labels':[]}

In [25]:
# read labels 
with open("../imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

In [26]:
for cate, filename in images.items():
    for f in filename:
        dir=os.path.join(path,cate,f)
        img=Image.open(dir)
        #print(dir)
        input_tensor=preprocess(img)
        input_batch = input_tensor.unsqueeze(0)

        if torch.cuda.is_available():
            input_batch= input_batch.to('cuda')
            model.to('cuda')

        with torch.no_grad():
            output = model(input_batch)
        
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        top10_prob, top10_catid = torch.topk(probabilities, 10)
#         typ_score=round(float(probabilities.max()*100),1)
        
        # Show top categories per image
        top10_prob, top10_catid = torch.topk(probabilities, 10)
        temp1 = {}
        temp2 = {}
        for i in range(top10_prob.size(0)):
            temp1[categories[top10_catid[i].item()]] = top10_prob[i].item()
            temp2[ltw['id'][top10_catid[i].item()]] = top10_prob[i].item()

        all_img['dir'].append(dir)
#         all_img['typicality_score'].append(typ_score)
        all_img['readable_labels'].append(temp1)
        all_img['id_labels'].append(temp2)
        if 'Mountain' in cate:
            all_img['type'].append('Mountain')
        if 'Beach' in cate:
            all_img['type'].append('Beach')
        if 'Mug' in cate:
            all_img['type'].append('Mug')
        if 'Banana' in cate:
            all_img['type'].append('Banana')
        if 'Car' in cate:
            all_img['type'].append('Car')
        if 'Plane' in cate:
            all_img['type'].append('Plane')
        if 'Lighthouse' in cate:
            all_img['type'].append('Lighthouse')
        if 'Church' in cate:
            all_img['type'].append('Church')

#         print('Typicality of ',f, ' = ',probabilities.max()*100)

In [27]:
all_img_df=pd.DataFrame(all_img)

In [28]:
all_img_df['type'].value_counts()

Church        16
Plane         16
Lighthouse    16
Car           16
Beach         16
Mountain      16
Mug           16
Banana        16
Name: type, dtype: int64

In [30]:
all_img_df.to_csv('googlenet_scores_wordnet_id.csv')

In [ ]:
# all_img_df['7_score']=round(all_img_df['typicality_score']*7/100,1)
# all_img_df.head()

In [ ]:
# import matplotlib.pyplot as plt
# score=[]
# for i in range(5):
#   dir=all_img_df['dir'][i]
#   score.append([all_img_df['typicality_score'][i], all_img_df['7_score'][i]])
#   img=Image.open(dir)
#   plt.imshow(img)
#   plt.title(score[i])
#   plt.show()

In [ ]:
# all_img_df.to_csv('google_withlabel.csv')

In [ ]:
# import matplotlib.pyplot as plt

# mount=all_img_df[all_img_df['type']=='Beach']
# plt.hist(mount['typicality_score'])